In [47]:
%load_ext autoreload
%autoreload 2

In [1]:
from sklearn import datasets

In [2]:
import numpy as np

In [3]:
iris = datasets.load_iris()

In [4]:
X = iris.data
y = iris.target

In [168]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

## とりあえず適当に分割してみる

In [170]:
np.unique(y, return_counts=True)

(array([0, 1, 2]), array([50, 50, 50]))

In [171]:
mask_divide = X[:, 1] > 2.5

In [172]:
y_upper = y[mask_divide]
y_lower = y[~mask_divide]

In [173]:
_, counts = np.unique(y_lower, return_counts=True)

In [174]:
prob = counts / len(y_lower)

### Gini係数を出してみる

$$\sum_{k=1}^K (1-p_k) p_k = 1 - \sum_{k=1}^K p_k^2$$

In [175]:
gini = 1 - (prob * prob).sum()

In [176]:
prob.sum()

1.0

In [177]:
gini

0.45983379501385047

### 分割点をスライドさせてgini係数が最大になる点を探そう

In [12]:
X_max = X[:, 1].max()
X_min = X[:, 1].min()

In [179]:
def gini(y):
    _, counts = np.unique(y, return_counts=True)

    prob = counts / len(y)
    
    return 1 - (prob * prob).sum()

In [180]:
gini(y)

0.6666666666666667

In [181]:
mask_divide = X[:, 1] > 2.5
y_upper = y[mask_divide]
y_lower = y[~mask_divide]

gini_divide = (gini(y_upper) * len(y_upper) + gini(y_lower) * len(y_lower)) / len(y)

In [182]:
gini_divide

0.6366680058925941

In [14]:
gini_org = gini(y)

In [15]:
list_gini = []
n_div = 100

X_div = np.array([X_min + (X_max - X_min) * i / n_div for i in range(0, n_div)])

for threshold in X_div:
    
    mask_divide = X[:, 1] > threshold
    y_upper = y[mask_divide]
    y_lower = y[~mask_divide]

    gini_divide = (gini(y_upper) * len(y_upper) + gini(y_lower) * len(y_lower)) / len(y)
    
    list_gini.append(gini_divide)

In [16]:
array_gini = np.array(list_gini)

In [17]:
i_div_opt = np.argmin(array_gini)

In [18]:
X_div[i_div_opt]

3.3200000000000003

In [19]:
array_gini[i_div_opt]

0.539743283106115

## 2/6 残タスク
- 別の特徴量で分割してgini係数を求める
- 刻み幅を一定ではなくて第何分点とかにする
- gini係数でなくエントロピー

# 2/7
- 分割点をデータそのものを一つずつ当ててみる
- 関数化して別の特徴量でも当てる
- 分割の様子をプロットにする

## 分割点をunique化したデータの値にしてみる

In [20]:
np.unique(X[:, 1])

array([2. , 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3,
       3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.4])

In [21]:
list_gini = []
X_div = np.unique(X[:, 1])

for threshold in X_div:
    
    mask_divide = X[:, 1] > threshold
    y_upper = y[mask_divide]
    y_lower = y[~mask_divide]

    gini_divide = (gini(y_upper) * len(y_upper) + gini(y_lower) * len(y_lower)) / len(y)
    
    list_gini.append(gini_divide)

In [22]:
array_gini = np.array(list_gini)

In [23]:
i_div_opt = np.argmin(array_gini)

In [24]:
X_div[i_div_opt]

3.3

In [25]:
array_gini[i_div_opt]

0.539743283106115

In [183]:
find_optimal_division(X[:, 1], y)

(3.3, 0.539743283106115)

### 関数にする

In [26]:
def find_optimal_division(x, y):
    list_gini = []
    x_unique = np.unique(x)

    for threshold in x_unique:

        mask_divide = x > threshold
        y_upper = y[mask_divide]
        y_lower = y[~mask_divide]

        gini_divide = (gini(y_upper) * len(y_upper) + gini(y_lower) * len(y_lower)) / len(y)

        list_gini.append(gini_divide)
        
    array_gini = np.array(list_gini)
    i_div_opt = np.argmin(array_gini)
    
    return x_unique[i_div_opt], array_gini[i_div_opt]


In [27]:
gini(y)

0.6666666666666667

## 全特徴量でやってみる

In [28]:
find_optimal_division(X[:, 1], y)

(3.3, 0.539743283106115)

In [30]:
[find_optimal_division(X[:, i], y) for i in range(0, X.shape[1])]

[(5.4, 0.4389063317634746),
 (3.3, 0.539743283106115),
 (1.9, 0.3333333333333333),
 (0.6, 0.3333333333333333)]

In [31]:
results = np.apply_along_axis(find_optimal_division, 0, X, y)

In [32]:
results

array([[5.4       , 3.3       , 1.9       , 0.6       ],
       [0.43890633, 0.53974328, 0.33333333, 0.33333333]])

In [33]:
arg_div = np.argmin(results[1])
x_div = results[0, arg_div]

In [34]:
arg_div, x_div

(2, 1.9)

## さらに分割してみる
- 不純度が一定以下になったら分割を止める必要がある
- ノードごとに持っておく量
    - 分割の通し番号（ノード番号）
    - 中間ノードなら，分割特徴量と閾値
    - 葉ノードなら，どのクラスに割り当てられるか

In [35]:
mask = X[:, arg_div] > x_div

In [36]:
y_upper = y[mask]
y_lower = y[~mask]

In [37]:
X_upper = X[mask]
X_lower = X[~mask]

In [38]:
results_upper = np.apply_along_axis(find_optimal_division, 0, X_upper, y_upper)
results_lower = np.apply_along_axis(find_optimal_division, 0, X_lower, y_lower)

In [39]:
results_upper

array([[6.1       , 2.4       , 4.7       , 1.7       ],
       [0.39313131, 0.46444444, 0.12646465, 0.11030596]])

In [40]:
results_lower

array([[4.3, 2.3, 1. , 0.1],
       [0. , 0. , 0. , 0. ]])

In [41]:
X_lower

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [42]:
y_lower

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [56]:
gini(y_lower)

0.0

In [43]:
y_upper

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [57]:
gini(y_upper)

0.5

# 2/7 残タスク
- 分割の様子のプロット
- 続けてどんどん分割を繰り返していくアルゴリズム
    - 不純度が一定以下になったら分割を止める必要がある

### めも
ノードごとに持っておく量
    - 分割の通し番号（ノード番号）
    - 中間ノードなら，分割特徴量と閾値
    - 葉ノードなら，どのクラスに割り当てられるか

# 2/17 続けて分割を繰り返す

### ワンセッション

In [44]:
from tree_modules import tree_base

In [49]:
tree_base.find_optimal_division(X[:, 2], y)

(1.9, 0.3333333333333333)

In [54]:
def divide_tree(X, y):

    results = np.apply_along_axis(tree_base.find_optimal_division, 0, X, y)

    arg_div = np.argmin(results[1])
    x_div = results[0, arg_div]

    return arg_div, x_div

In [55]:
divide_tree(X, y)

(2, 1.9)

### 続けるかどうかを判断しながら繰り返す

In [165]:
def go_on_dividing(X, y, depth=0, div_set=None,
                   threshold_gini=0.05, min_node_size=5, max_depth=3):
    
    global i
    if div_set is None:
        div_set = []
        
    depth += 1
    
    arg_div, x_div = divide_tree(X, y)
    
    print("=== node {} (depth {}): arg_div -> {}, x_div -> {} ===".format(i, depth, arg_div, x_div))
    
    mask = X[:, arg_div] > x_div
    X_upper, X_lower = X[mask], X[~mask]
    y_upper, y_lower = y[mask], y[~mask]
    
    gini_lower = gini(y_lower)
    gini_upper = gini(y_upper)
    
    list_divided = [(X_lower, y_lower, gini_lower), (X_upper, y_upper, gini_upper)]
    
    for ul, divided in enumerate(list_divided):
        i +=1
        div_set_tmp = div_set.copy()
        div_set_tmp.append((depth, i, arg_div, x_div, ul))
        
        X_i, y_i, gini_i = divided
        if gini_i > threshold_gini and len(y_i)>min_node_size and depth+1 <= max_depth:
            go_on_dividing(X_i, y_i, depth=depth, div_set=div_set_tmp)
        else:
            # なんらかの停止処理
            print(div_set_tmp, np.bincount(np.array(y_i)).argmax())

In [166]:
i = 0
go_on_dividing(X, y)

=== node 0 (depth 1): arg_div -> 2, x_div -> 1.9 ===
[(1, 1, 2, 1.9, 0)] 0
=== node 2 (depth 2): arg_div -> 3, x_div -> 1.7 ===
=== node 3 (depth 3): arg_div -> 2, x_div -> 4.9 ===
[(1, 2, 2, 1.9, 1), (2, 3, 3, 1.7, 0), (3, 4, 2, 4.9, 0)] 1
[(1, 2, 2, 1.9, 1), (2, 3, 3, 1.7, 0), (3, 5, 2, 4.9, 1)] 2
[(1, 2, 2, 1.9, 1), (2, 6, 3, 1.7, 1)] 2
